## Code to install MILC and QUDA on Summit
Sept 16, 2021

In [2]:
import os, glob
import subprocess as sp
import numpy as np
import shutil


In [3]:
from ipywidgets import *

In [4]:
start_dir=os.getcwd()
print(start_dir)

/autofs/nccs-svm1_home1/venkitesh/projects/lqcd/multigrid_lattice/Summit_scripts


In [5]:
dict_pars={}
dict_pars['build_dir']='/gpfs/alpine/lgt104/proj-shared/ayyar/builds_summit/install_March_14_2022'
dict_pars['mgrid']='-DQUDA_MULTIGRID=ON'
dict_pars['run_dir']='/gpfs/alpine/lgt104/proj-shared/ayyar/runs_summit/runs_March14_2022'
# dict_pars['mgrid']=''

## Create upper level directories
if not os.path.exists(dict_pars['build_dir']):
    os.makedirs(dict_pars['build_dir']+'/QUDA')
    os.makedirs(dict_pars['build_dir']+'/MILC')

In [6]:
### Setup env
env_bash_strg='''#!/bin/bash
#############

module purge
module load DefApps
module unload xalt/1.2.1
#module load cuda/11.3.1
module load cuda/11.0.3
#module load gcc/7.5.0
module load gcc/9.3.0
module load git/2.31.1
module load cmake/3.20.2
module load nsight-systems/2021.3.1.54
module load nsight-compute/2021.2.1
module load gdrcopy/2.2
module list


export LD_LIBRARY_PATH="{build_dir}/QUDA/build/usqcd/lib:${{LD_LIBRARY_PATH}}"
'''.format(**dict_pars)


In [7]:

build_bash_strg='''#!/bin/bash
#############
WORKDIR={build_dir}

#############
## Source environment
source ${{WORKDIR}}/setup_env.sh

############
## Build QUDA
QUDA_SRC=${{WORKDIR}}/QUDA/src/quda
QUDA_BUILD=${{WORKDIR}}/QUDA/build

pushd .
if [ ! -d $QUDA_SRC ]
then
  git clone --branch develop https://github.com/lattice/quda $QUDA_SRC # clone QUDA to the desired directory
else
  cd $QUDA_SRC; git pull; cd ..
fi

if [ ! -d $QUDA_BUILD ]
then
  mkdir $QUDA_BUILD
else
  cd $QUDA_BUILD
  rm -rf *
fi

cd $QUDA_BUILD
cmake -DCMAKE_BUILD_TYPE=RELEASE -DQUDA_DIRAC_DEFAULT_OFF=ON -DQUDA_DIRAC_STAGGERED=ON -DQUDA_GPU_ARCH=sm_70 -DQUDA_GAUGE_ALG=ON -DQUDA_GAUGE_TOOLS=ON -DQUDA_DOWNLOAD_USQCD=ON -DQUDA_QIO=ON -DQUDA_QMP=ON {mgrid} $QUDA_SRC
nice make -j 4
make install
popd

############
## Build MILC

MILCDIR=${{WORKDIR}}/MILC/milc_qcd
if [ -d $MILCDIR ]
then
  rm -rf $MILCDIR
fi
git clone --branch develop https://github.com/milc-qcd/milc_qcd ${{MILCDIR}}
cd ${{MILCDIR}}/ks_spectrum && cp ${{MILCDIR}}/Makefile ${{MILCDIR}}/ks_spectrum
export PATH_TO_CUDA=/sw/summit/cuda/11.0.3 # NEED TO MAKE THIS MORE ROBUST TO DIFFERENT CUDA VERSIONS, ASSUMES 11.0.3
export USQCD_BUILD=${{QUDA_BUILD}}/usqcd

PATH_TO_CUDA=$PATH_TO_CUDA PATH_TO_QUDA=$QUDA_BUILD PATH_TO_QMP=$USQCD_BUILD PATH_TO_QIO=$USQCD_BUILD POWER9=1 MULTIGRID=1 ./compile_ks_spectrum_quda.sh

cd $WORKDIR
'''.format(**dict_pars)


In [8]:
loc1=start_dir+'/install_scripts'
loc2=dict_pars['build_dir']
loc1,loc2

('/autofs/nccs-svm1_home1/venkitesh/projects/lqcd/multigrid_lattice/Summit_scripts/install_scripts',
 '/gpfs/alpine/lgt104/proj-shared/ayyar/builds_summit/install_March_14_2022')

In [9]:
pwd

'/autofs/nccs-svm1_home1/venkitesh/projects/lqcd/multigrid_lattice/Summit_scripts'

In [10]:

fname1='setup_env.sh'
with open (loc1+'/'+fname1,'w') as f: f.write(env_bash_strg)

fname2='build.sh'
with open (loc1+'/'+fname2,'w') as f: f.write(build_bash_strg)

print(fname1,fname2)


setup_env.sh build.sh


In [11]:
%%bash -s "$fname1" "$fname2" "$loc1" "$loc2" ## Use python variables in bash
# cat $1
# echo "-----------------"
# cat $2

chmod +x $3/$1
chmod +x $3/$2

cp $3/$1 $4/$1
cp $3/$2 $4/$2


In [12]:
# %%bash -s "$filename" ## Use python variable in bash
# # module load cgpu
# ./$filename > op.out 2>&1

In [13]:
cat install_scripts/build.sh

#!/bin/bash
#############
WORKDIR=/gpfs/alpine/lgt104/proj-shared/ayyar/builds_summit/install_March_14_2022

#############
## Source environment
source ${WORKDIR}/setup_env.sh

############
## Build QUDA
QUDA_SRC=${WORKDIR}/QUDA/src/quda
QUDA_BUILD=${WORKDIR}/QUDA/build

pushd .
if [ ! -d $QUDA_SRC ]
then
  git clone --branch develop https://github.com/lattice/quda $QUDA_SRC # clone QUDA to the desired directory
else
  cd $QUDA_SRC; git pull; cd ..
fi

if [ ! -d $QUDA_BUILD ]
then
  mkdir $QUDA_BUILD
else
  cd $QUDA_BUILD
  rm -rf *
fi

cd $QUDA_BUILD
cmake -DCMAKE_BUILD_TYPE=RELEASE -DQUDA_DIRAC_DEFAULT_OFF=ON -DQUDA_DIRAC_STAGGERED=ON -DQUDA_GPU_ARCH=sm_70 -DQUDA_GAUGE_ALG=ON -DQUDA_GAUGE_TOOLS=ON -DQUDA_DOWNLOAD_USQCD=ON -DQUDA_QIO=ON -DQUDA_QMP=ON -DQUDA_MULTIGRID=ON $QUDA_SRC
nice make -j 4
make install
popd

############
## Build MILC

MILCDIR=${WORKDIR}/MILC/milc_qcd
if [ -d $MILCDIR ]
then
  rm -rf $MILCDIR
fi
git clone --branch develop https://github.com/milc-qcd/milc_qcd ${MI

### Run the build
./build.sh 2>&1 | tee op.out




### Run actual code

In [16]:
# target_dir=dict_pars['run_dir']
# src_dir='/gpfs/alpine/lgt104/proj-shared/weinbe2/for-venkitesh/'
# exec_file=dict_pars['build_dir']+'/MILC/milc_qcd/ks_spectrum/ks_spectrum_hisq'
# if not os.path.exists(target_dir):
#     os.makedirs(target_dir)


In [17]:
# %%bash -s "$src_dir" "$target_dir" "$exec_file" ## Use python variables in bash

# cp -r $1/* $2/
# mkdir $2/rand
# ln -s $3 $2/

In [18]:
# #!/bin/bash -v
# #BSUB -P lgt104
# #BSUB -W 3:00
# #BSUB -nnodes 108
# #BSUB -J fpi000569
# #BSUB -o kpp_spectrumOut.%J
# #BSUB -e kpp_spectrumErr.%J
# #BSUB -cn_cu 'maxcus=6'
# #BSUB -alloc_flags "smt4"

# # submit with 
# # bsub run.lsf

# nodes=108

# PROJ=/gpfs/alpine/lgt104/proj-shared/ayyar/builds_lqcd/run_code
# source /gpfs/alpine/lgt104/proj-shared/ayyar/builds_lqcd/install_Sept30_2021/setup_env.sh


# executable=${PROJ}/ks_spectrum_hisq
# input=input.kpp
# output=output-pretune.kpp

# node_geom="3 2 6 18"
# io_geom="${node_geom}"
# runargs="-qmp-geom ${node_geom} -qmp-alloc-map 3 2 1 0 -qmp-logic-map  3 2 1 0"

# export QUDA_RESOURCE_PATH=${PROJ}/tunecache # location of QUDA tunecache file
# mkdir -p $QUDA_RESOURCE_PATH
# export QUDA_ENABLE_GDR=1
# export QUDA_MILC_HISQ_RECONSTRUCT=13
# export QUDA_MILC_HISQ_RECONSTRUCT_SLOPPY=9

# #export QUDA_ENABLE_DEVICE_MEMORY_POOL=0
# #export QUDA_ENABLE_MANAGED_MEMORY=1
# #export QUDA_ENABLE_MANAGED_PREFETCH=1

# echo "START_RUN: `date`" >> $output
# echo "EXECUTABLE =  $executable" >> $output
# ls -l $executable >> $output
# echo "RUNARGS = $runargs" >> $output

# export OMP_NUM_THREADS=7
# export APP="$executable $runargs $input $output"
# echo ${APP} >> ${output}
# cmd="jsrun --smpiargs="'"-gpu"'" -n $nodes -r 1 -g 6 -a 6 -c 42 -dpacked -b packed:7 -l gpu-cpu ./bind-6gpu.sh"

# echo COMMAND: $cmd >> $output
# $cmd

# echo "FINISH_RUN: `date`" >> $output

In [19]:
pwd

'/autofs/nccs-svm1_home1/venkitesh/projects/lqcd/multigrid_lattice'

In [20]:
! cp /gpfs/alpine/lgt104/proj-shared/ayyar/builds_lqcd/run_mgrid/input-mg.kpp .

In [21]:
ls

Data_extract/                exec_file@    install_scripts/  run_scripts/
build_install_scripts.ipynb  input-mg.kpp  older_scripts/
